In [58]:
lipidname = "OIPC"
tail = "CDDC CDCC" 
link = "G G"
head = "C P"

membrane="testmembrane"
insane="./insane+SF.py"
mdparams="./test.mdp"
martinipath="./martini.ff"

# Cleaning up intermediate files from previous runs
!rm -f *#*
!rm -f *step*
!rm -f {membrane}*

In [59]:
print("Create itp")
!python {martinipath}/lipid-martini-itp-v06.py -o {martinipath}/martini_v2_omaralab.itp -alname {lipidname} -alhead '{head}' -allink '{link}' -altail '{tail}'
print("Done")

Create itp
The Martini lipid itp generator version 0.6  Args are: -o ./martini.ff/martini_v2_omaralab.itp -alname OIPC -alhead 'C P' -allink 'G G' -altail 'CDDC CDCC'
Done


In [62]:
# build a simple membrane  to visualize this species
insane=!python2 {insane} -o {membrane}.gro -p {membrane}.top -d 0 -x 3 -y 3 -z 3 -sol PW -center -charge 0 -u {lipidname}:1 -l {lipidname}:1

# use GMX to convert GRO file into a PDB
output = !gmx editconf -f {membrane}.gro -o  {membrane}.pdb

# create a file to automate rendering an image of the protein, 
# display projection Orthographic
# setting "bead" style, 
# setting the background to white,
# turning off the XYZ axes,  
# and taking a snapshot 

with open("output.tcl", "w") as file:
    file.write("mol modselect 0 0 resid 1\n")
    file.write("rotate x by 0\n")
    file.write("rotate y by 120\n")
    file.write("rotate z by 90\n")
    file.write("translate by 0 -1.1 0\n")
    file.write("display projection Orthographic\n")
    file.write("rotate x by 30\n")
    file.write("rotate y by 20\n")
    file.write("mol modstyle 0 0 VDW 0.25 12\n")
    file.write("axes location Off\n")
    file.write("color Display Background white\n")
    file.write("scale by 3.5\n")
    file.write("set sel [atomselect 0 all]\n")
    file.write("set atomlist [$sel list]\n")
    file.write("foreach {atom} $atomlist {\n")
    file.write("    set atomlabel [format \"%d/%d\" 0 $atom]\n")
    file.write("    label add Atoms $atomlabel\n")
    file.write("}\n")
    file.write("color Labels Atoms blue\n")
    file.write("label textthickness 2\n")
    file.write("label textsize 2\n")
    file.write("rotate z by 30\n")
    #file.write("translate by 0.5 0 0\n")
    file.write("render snapshot image.tga {convert image.tga jpg:image.jpg; rm image.tga}\n")
    file.write("exit")
    
# run VMD to render the image 
output = !vmd {membrane}.pdb -e output.tcl 

# clean up temporary files
#!rm output.tcl

import random
counter = random.randint(0,2e9)

from IPython.display import HTML, display
display(HTML('<h1>{}</h1><br/><img src="image.jpg?{}" alt="Image of {}" width="300">'.format(lipidname,counter,lipidname)))

In [64]:
import os #Operating system specific commands
import re #Regular expression library

print("Test")
print("Grompp")
grompp = !gmx grompp -f {mdparams} -c {membrane}.gro -p {membrane}.top -o  {membrane}.tpr
success=True
for line in grompp:
    if re.search("ERROR", line):
        success=False
    if re.search("Fatal error", line):
        success=False
    if not success:
        print(line)
    
if success:
    print("Run")
    !export GMX_MAXCONSTRWARN=-1
    !export GMX_SUPPRESS_DUMP=1
    run = !gmx mdrun -v -deffnm {membrane}
    summary=""
    logfile = membrane+".log"
    if not os.path.exists(logfile):
        print("no log file")
        print("== === ====")
        for line in run:
            print(line)
    else:
        try:
            file = open(logfile, "r")
            fe = False
            for line in file:
                if fe:
                    success=False
                    summary=line
                elif re.search("^Steepest Descents.*converge", line):
                    success=True
                    summary=line
                    break
                elif re.search("Fatal error", line):
                    fe = True
        except IOError as exc:
            sucess=False;
            summary=exc;
    if success:
        print("Success")
    else:
        print(summary)

Test
Grompp
ERROR 1 [file testmembrane.top, line 19]:
  ERROR: The cut-off length is longer than half the shortest box vector or
  longer than the smallest box diagonal element. Increase the box size or
  decrease rlist.


Removing all charge groups because cutoff-scheme=Verlet

-------------------------------------------------------
Program:     gmx grompp, version 2018.1
Source file: src/gromacs/gmxpreprocess/grompp.cpp (line 1991)

Fatal error:
There was 1 error in input file(s)

For more information and tips for troubleshooting, please check the GROMACS
website at http://www.gromacs.org/Documentation/Errors
-------------------------------------------------------
